---
Import, process, and clean the data
---
---


In [5]:
import pandas as pd
import numpy as np 

# Load in the data 
#df = pd.read_csv('rspct.tsv', sep="\t")
df = pd.read_csv(r"C:\Users\Z Dubs\lambda\sr_predict\reddit_data\rspct.tsv", sep="\t")

In [6]:
df.shape

(1013000, 4)

In [7]:
# Get rid of ambiguous data that can decrese model efficiency 
df = df.drop(['id'], axis=1)

# Combine 'title' and 'selftext' columns to enrich data
df["full_text"] = df["title"] + " " + df["selftext"]
df.head()

,subreddit,title,selftext,full_text
0,talesfromtechsupport,Remember your command line switches...,"Hi there, <lb>The usual. Long time lerker, fi...",Remember your command line switches... Hi ther...
1,teenmom,"So what was Matt ""addicted"" to?",Did he ever say what his addiction was or is h...,"So what was Matt ""addicted"" to? Did he ever sa..."
2,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,No Club Colors Funny story. I went to college ...
3,ringdoorbell,"Not door bell, but floodlight mount height.",I know this is a sub for the 'Ring Doorbell' b...,"Not door bell, but floodlight mount height. I ..."
4,intel,Worried about my 8700k small fft/data stress r...,"Prime95 (regardless of version) and OCCT both,...",Worried about my 8700k small fft/data stress r...


In [8]:
# Extract 5 percent of the dataset, even .1 of the data yields memory errors
df = df.sample(frac=0.05, random_state=7)

In [9]:
from bs4 import BeautifulSoup
# Function to clean html tags
def soup(text):
    soup = BeautifulSoup(text, 'html.parser')
    clean_text = soup.get_text()
    return clean_text

In [10]:
df['clean_text'] = df['full_text'].apply(soup)
df.head()

,subreddit,title,selftext,full_text,clean_text
929565,OnePunchMan,[Spoilers] Black Sperm's cell stock,"We all know Black Sperm is op, but let's revie...",[Spoilers] Black Sperm's cell stock We all kno...,[Spoilers] Black Sperm's cell stock We all kno...
100277,devops,PSA: Zookeeper 3.5 beta issues (can break Apac...,If anyone is looking into deploying Zookeeper ...,PSA: Zookeeper 3.5 beta issues (can break Apac...,PSA: Zookeeper 3.5 beta issues (can break Apac...
707395,namenerds,How do you design a sibset?,"Say you pick the first child's name, and it's ...",How do you design a sibset? Say you pick the f...,How do you design a sibset? Say you pick the f...
374556,twinpeaks,[S3E16] Plotlines still to be resolved before ...,With only two hours to go I wanted to get this...,[S3E16] Plotlines still to be resolved before ...,[S3E16] Plotlines still to be resolved before ...
503596,FidgetSpinners,Why do teachers like to take Fidget Spinners?,I only see one situation. I can see when peopl...,Why do teachers like to take Fidget Spinners? ...,Why do teachers like to take Fidget Spinners? ...


In [11]:
# Taking a look at value count of target 
df['subreddit'].value_counts()

AskEconomics          79
AcademicPsychology    70
Sneakers              70
adderall              69
gravityfalls          68
                      ..
RWBY                  34
latin                 33
RocketLeague          33
civilengineering      33
CryptoKitties         31
Name: subreddit, Length: 1013, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split

# We'll do a 80/20 train/test split and stratify the target 
train, test = train_test_split(df, test_size=0.2, stratify=df["subreddit"])

# Sanity check 
train.shape, test.shape

((40520, 5), (10130, 5))

In [13]:
# Assign data to features/target
X_train = train["clean_text"]
X_test = test["clean_text"]

y_train = train["subreddit"]
y_test = test["subreddit"]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40520,)
(10130,)
(40520,)
(10130,)


In [14]:
y_train.head()

50695         Epilepsy
995704         amiugly
225816             ftm
848525    ringdoorbell
724584         thesims
Name: subreddit, dtype: object

In [15]:
# Sanity check v2
y_train.value_counts()

AskEconomics          63
AcademicPsychology    56
Sneakers              56
adderall              55
osugame               54
                      ..
RWBY                  27
latin                 26
RocketLeague          26
civilengineering      26
CryptoKitties         25
Name: subreddit, Length: 1013, dtype: int64

---
Use Label Encoding on the target 
---
---

In [16]:
from sklearn.preprocessing import LabelEncoder

# Instantiate LabelEncoder
encoder = LabelEncoder()
# Fit the encoder
encoder.fit(y_train)
# Transform on train and test 
y_train = encoder.transform(y_train)
y_test  = encoder.transform(y_test)

---
Create a pipeline equipped with countvectorizer, tfidf-transformer, and a SGDClassifier model
---
---

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
    ])

In [18]:
text_clf.fit(X_test, y_test)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000,

In [19]:
test_post = np.array(["I love getting new sneakers; Jordan's, Nike, Addidas, custom footwear. Wanting to see what people think of these fresh kicks. I love the laces, contour of the shoe, more generic sneaker and shoe terms"])
text_clf.predict(test_post)

array([329])

In [20]:
df.iloc[329][0]

'tattoo'

In [167]:
from sklearn.metrics import classification_report

text_clf.fit(X_train,y_train)
predictions = text_clf.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.64      0.90      0.75        10
           1       0.60      0.38      0.46         8
           2       0.62      0.50      0.56        10
           3       0.71      0.56      0.63         9
           4       0.83      0.45      0.59        11
           5       0.91      0.83      0.87        12
           6       0.75      0.50      0.60        12
           7       0.52      1.00      0.69        11
           8       0.91      0.91      0.91        11
           9       0.50      0.56      0.53         9
          10       0.57      0.40      0.47        10
          11       0.69      0.92      0.79        12
          12       0.54      0.78      0.64         9
          13       0.58      0.50      0.54        14
          14       0.50      0.67      0.57         9
          15       0.56      0.50      0.53        10
          16       0.80      0.89      0.84         9
          17       0.50    

E:\Anaconda\envs\U4-S1-NLP\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
from sklearn import metrics
y_train_pred = text_clf.predict(X_train)
print (metrics.accuracy_score(y_train, y_train_pred))

0.5340078973346496


---
Alright, we can make a prediction and see the top result, lets make a function that returns the top 5 results
---
---

In [24]:
def get_predictions(post, num_answers=5):
  """ takes a potential post and returns the top options """

  preds = pd.Series(text_clf.decision_function(post)[0])

  preds.index = text_clf.classes_

  preds = preds.sort_values(ascending=False)

  return preds[:5]

In [25]:
get_predictions(test_post)

329   -0.401971
248   -0.958687
137   -0.985770
647   -0.987266
209   -0.989961
dtype: float64

In [26]:
df.iloc[324]

subreddit                                       DBZDokkanBattle
title                               dokkan battlefield level 16
selftext      how the fuck do you beat this level? ive sent ...
full_text     dokkan battlefield level 16 how the fuck do yo...
clean_text    dokkan battlefield level 16 how the fuck do yo...
Name: 711479, dtype: object

In [27]:
nba_post = [ """
               LeBron James and Kobe Bryant and both great NBA players and we should stop comparing them.
                """]

In [28]:
get_predictions(nba_post)

738   -0.870125
982   -0.873794
772   -0.933504
457   -0.964201
989   -0.967183
dtype: float64

In [29]:
df.iloc[738]

subreddit                                                Ripple
title                              Quick paper wallet questions
selftext      I plan on buying Ripple soon and storing it in...
full_text     Quick paper wallet questions I plan on buying ...
clean_text    Quick paper wallet questions I plan on buying ...
Name: 254102, dtype: object

---
Testing pipeline with a couple other models really quickly, first RFC
---
---

In [119]:
from sklearn.ensemble import RandomForestClassifier
text_clf1 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier()),
    ])

text_clf1.fit(X_test, y_test)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

In [120]:
test_post = np.array(["I love getting new sneakers; Jordan's, Nike, Addidas, custom footwear. Wanting to see what people think of these fresh kicks"])
text_clf1.predict(test_post)

array([329])

In [121]:
from sklearn import metrics
y_train_pred1 = text_clf1.predict(X_train)
print (metrics.accuracy_score(y_train, y_train_pred1))

0.4590819348469891


---
Ok, not optimal, but we got a baseline. Lets pickle the SGD model and try something a little more complex 
---
---

In [161]:
import pickle

# Save the Model to file in the current working directory
Pkl_Filename = "Baseline_SGD_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(text_clf, file)

---
The pickled model is too big! Have to break up the pipeline and pickle individually
---
---

In [30]:
# This didn't work, but an attempt was made so we'll leave it in here
#def picklizer(to_pickle, filename, path):
    """
    Creates a pickle file.
    
    Parameters
    ----------
    to_pickle : Python object
        The trained / fitted instance of the 
        transformer or model to be pickled.
    filename : string
        The desired name of the output file,
        not including the '.pkl' extension.
    path : string or path-like object
        The path to the desired output directory.
    """
    import os
    import pickle

    # Create the path to save location
    picklepath = os.path.join(path, filename)

    # Use context manager to open file
    with open(picklepath, "wb") as p:
        pickle.dump(to_pickle, p)

In [33]:
#vect = text_clf.named_steps['vect']
#tfidf = text_clf.named_steps['tfidf']
#clf = text_clf.named_steps['clf']

In [35]:
#filepath = "models/"

# Export vectorizer as pickle
#picklizer(vect, "vec_01.pkl", filepath)

# Export transformer as pickle
#picklizer(tfidf, "tfidf_01.pkl", filepath)

# Export sgd model as pickle
#picklizer(clf, "clf_01.pkl", filepath)

---
Now lets load the pickled model and try to run predictions
---
---

In [162]:
with open(Pkl_Filename, 'rb') as file:  
    Baseline_SGD_Model = pickle.load(file)

Baseline_SGD_Model

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000,

In [163]:
Baseline_SGD_Model.fit(X_test, y_test)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000,

In [165]:
Baseline_SGD_Model.predict(test_post)

array([329])

In [166]:
from sklearn import metrics
y_train_pred = Baseline_SGD_Model.predict(X_train)
print (metrics.accuracy_score(y_train, y_train_pred))

0.5342546890424482


---
The only way we could get the pickled model on github was compressing the file with 7zip, lets try to unzip it 
---
---

In [68]:
pip install py7zr

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [70]:
import py7zr

archive = py7zr.SevenZipFile('Baseline_SGD_Model.7z', mode='r')
archive.extractall(path="/tmp")
archive.close()